In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
//import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.apache.spark::spark-mllib:2.1.0` // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
import Resolvers._

import $exclude.$                        , $ivy.$                            // for cleaner logs
//import $profile.`hadoop-2.6`

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                     // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

import Resolvers._

In [2]:
import $ivy.`org.vegas-viz::vegas:0.3.9`
import vegas._
import vegas.data.External._

import $ivy.$                           

import vegas._

import vegas.data.External._

In [3]:
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "isarn project",
  "https://dl.bintray.com/isarn/maven/",
  MavenPattern,
  true
)
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "Will Benton",
  "https://dl.bintray.com/willb/maven/",
  MavenPattern,
  true
)

In [4]:
import $ivy.`org.isarnproject::isarn-sketches:0.0.3.rc1`
import org.isarnproject.sketches._

import $ivy.$                                           

import org.isarnproject.sketches._

In [5]:
import $ivy.`com.redhat.et::silex:0.1.2`

import $ivy.$                           

In [6]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.classification.ClassificationModel
import org.apache.spark.mllib.linalg.DenseVector

import com.redhat.et.silex.util.vectors.implicits._

import org.apache.spark.rdd.RDD

import org.apache.spark.mllib.tree.RandomForest

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}

import org.apache.spark.mllib.evaluation.MulticlassMetrics

import org.apache.spark.mllib.regression.LabeledPoint

import org.apache.spark.mllib.classification.ClassificationModel

import org.apache.spark.mllib.linalg.DenseVector


import com.redhat.et.silex.util.vectors.implicits._

In [7]:
val spark = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("spark://notebook:7077")
  .appName("notebook")
  .getOrCreate()
def sc = spark.sparkContext

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


spark: SparkSession = org.apache.spark.sql.SparkSession@6ca1dbcf
defined function sc

In [8]:
def loadCSV(fname: String) = spark.read
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .option("mode", "DROPMALFORMED")
  .load(fname)

defined function loadCSV

In [9]:
implicit class AddSampling(td: TDigest) extends Serializable {
    import org.isarnproject.collections.mixmaps.nearest.Cover
    def cdfDiscrete(x: Double): Double = {
        td.clusters.coverR(x).l.fold(0.0){ case(x,_) => td.clusters.prefixSum(x) / td.clusters.sum }
    }
    def cdfDiscreteInverse(q: Double): Double = {
        td.clusters.mCover(q * td.clusters.sum).map(_.data.key) match {
            case Cover(_, Some(x)) => x
            case Cover(Some(x), None) => x
        }
    }
    def sample: Double = {
        val clust = td.clusters
        td.nclusters match {
            case 0 => 0.0
            case n if (n <= td.maxDiscrete) => cdfDiscreteInverse(scala.util.Random.nextDouble)
            case _ => td.cdfInverse(scala.util.Random.nextDouble)
        }
    }
}

defined class AddSampling

In [10]:
def tdSketchFV(fv: RDD[Array[Double]]): Vector[TDigest] = {
    val tds = fv.aggregate(Array.empty[TDigest])(
        (tdv, xv) => {
            if (tdv.isEmpty) {
                Array.tabulate(xv.length) { j => TDigest.empty(maxDiscrete=50) + xv(j) }
            } else {
                for { j <- 0 until xv.length } { tdv(j) += xv(j) }
                tdv
            }
        },
        (tdv1, tdv2) => {
            if (tdv1.isEmpty) tdv2 else {
                for { j <- 0 until tdv1.length } { tdv1(j) ++= tdv2(j) }
                tdv1
            }
        }
    )
    tds.toVector
}

defined function tdSketchFV

In [11]:
val trainCSV = loadCSV("/data/tox21/tox21_dense_train.csv")
val testCSV = loadCSV("/data/tox21/tox21_dense_test.csv")
val trainLabelsCSV = loadCSV("/data/tox21/tox21_labels_train.csv")
val testLabelsCSV = loadCSV("/data/tox21/tox21_labels_test.csv")

trainCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]
testCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]
trainLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]
testLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]

In [12]:
def csvToPairs(csv: DataFrame) =
    csv.rdd.map(_.toSeq)
    .map(v => (v.head.asInstanceOf[String], v.tail.toVector.map(_.asInstanceOf[String].toDouble)))
def csvToLP(tlab: String, labCSV: DataFrame, fvCSV: DataFrame) = {
    val labCol = labCSV.select("_c0", tlab)
    val labPairs = labCol.filter(labCol(tlab) =!= "NA")
        .rdd.map(_.toSeq).map { s => (s(0).asInstanceOf[String], s(1).asInstanceOf[String].toDouble) }
    val fvPairs = csvToPairs(fvCSV)
    val lp = labPairs.join(fvPairs).map { case (_, (lab, feats)) => LabeledPoint(lab, feats.toSpark) }
    lp
}

defined function csvToPairs
defined function csvToLP

In [13]:
trainLabelsCSV.columns.mkString(" ")

res12: String = "_c0 NR.AhR NR.AR NR.AR.LBD NR.Aromatase NR.ER NR.ER.LBD NR.PPAR.gamma SR.ARE SR.ATAD5 SR.HSE SR.MMP SR.p53"

In [14]:
val trainLP = csvToLP("`NR.AhR`", trainLabelsCSV, trainCSV)
val testLP = csvToLP("`NR.AhR`", testLabelsCSV, testCSV)

trainLP: RDD[LabeledPoint] = MapPartitionsRDD[43] at map at cmd11.sc:9
testLP: RDD[LabeledPoint] = MapPartitionsRDD[59] at map at cmd11.sc:9

In [15]:
val lrModel = new LogisticRegressionWithLBFGS().run(trainLP)

lrModel: LogisticRegressionModel = org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 0.0, numFeatures = 801, numClasses = 2, threshold = 0.5

In [16]:
def variableImportance(
    model: ClassificationModel,
    lp: RDD[LabeledPoint],
    sc: SparkContext = spark.sparkContext): Array[Double] = {

    // sketch each feature using t-digest
    val fvtd = tdSketchFV(lp.map(_.features.toArray))
    val m = fvtd.length
    val n = fvtd.head.clusters.sum
    // broadcast the sketches
    val tdBC = sc.broadcast(fvtd)
    // count how many predictions differ, by feature,
    // when a randomized feature value is used
    val dcounts = lp.aggregate(Array.fill(m)(0L))(
        (counts, lpt) => {
            val LabeledPoint(lab, feats) = lpt
            val refpred = model.predict(feats)
            val fa = feats.toArray
            val fv = new DenseVector(fa)
            for { j <- 0 until m } {
                val t = fa(j)
                fa(j) = tdBC.value(j).sample
                val pred = model.predict(fv)
                fa(j) = t
                if (pred != refpred) counts(j) += 1
            }
            counts
        },
        (counts1, counts2) => {
            for { j <- 0 until m } {
                counts1(j) += counts2(j)
            }
            counts1
        }
    )
    // report differences as a fraction of total samples
    dcounts.map(c => c.toDouble / n)
}

defined function variableImportance

In [17]:
val rawVI = variableImportance(lrModel, trainLP)

rawVI: Array[Double] = Array(
  0.005449591280653951,
  0.005804999407653121,
  0.0037910200213244877,
  0.019784385736287168,
  0.0022509181376614146,
  0.002013979386328634,
  0.007108162539983414,
  0.00473877502665561,
  0.00473877502665561,
  0.006989693164317024,
  0.0015401018836630732,
...

In [18]:
val viPairs = trainCSV.columns.tail.zip(rawVI).sortBy { case (_, vi) => -vi}

viPairs: Array[(String, Double)] = Array(
  ("MRVSA9", 0.0363700983295818),
  ("slogPVSA9", 0.03281601705959009),
  ("slogPVSA10", 0.03257907830825731),
  ("RASA", 0.028314180784267267),
  ("UI", 0.025589385143940293),
  ("RPSA", 0.02476009951427556),
  ("PEOEVSA7", 0.024167752635943607),
  ("MRVSA2", 0.023575405757611658),
  ("FNSA3", 0.022627650752280536),
  ("slogPVSA11", 0.022509181376614146),
  ("PNSA3", 0.022390712000947756),
...

In [19]:
viPairs.take(10)

res18: Array[(String, Double)] = Array(
  ("MRVSA9", 0.0363700983295818),
  ("slogPVSA9", 0.03281601705959009),
  ("slogPVSA10", 0.03257907830825731),
  ("RASA", 0.028314180784267267),
  ("UI", 0.025589385143940293),
  ("RPSA", 0.02476009951427556),
  ("PEOEVSA7", 0.024167752635943607),
  ("MRVSA2", 0.023575405757611658),
  ("FNSA3", 0.022627650752280536),
  ("slogPVSA11", 0.022509181376614146)
)

In [20]:
viPairs.takeRight(10)

res19: Array[(String, Double)] = Array(
  ("RDFE14", 0.0),
  ("RDFE20", 0.0),
  ("RDFE22", 0.0),
  ("RDFE29", 0.0),
  ("RDFP25", 0.0),
  ("RDFU12", 0.0),
  ("RDFU29", 0.0),
  ("RDFU7", 0.0),
  ("RNCS", 0.0),
  ("Vm", 0.0)
)

In [21]:
viPairs.count { case (_, vi) => vi > 0.01 }

res20: Int = 86